In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import sys
import random
import IPython

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.utils import shuffle
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

from matplotlib.pyplot import imread
from cv2 import resize
import cv2

from keras import layers
from keras.layers import Input, Add, Dense,Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from sklearn.preprocessing import OneHotEncoder

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

random.seed = 2
np.random.seed = 2
tf.seed = 2
tf.random.set_seed(2)

# Train Data Read

In [ ]:
dataset_images_location = "../input/aptos2019-blindness-detection/train_images"
dataset_groundtruth_location = "../input/aptos2019-blindness-detection/train.csv"

df = pd.read_csv(dataset_groundtruth_location)
df_ = pd.DataFrame();
df_['filename'] = df['id_code']+'.png'
df_['grade'] = df['diagnosis']

labels = np.array(df_['grade']).reshape(-1, 1)
enc = OneHotEncoder(categories='auto', drop=None, sparse=False, dtype = np.int, handle_unknown='error')
labels = enc.fit_transform(labels)

a = []
for i in range(len(labels)):
    a.append(str(labels[i]))
    
df_['class'] = a

df = df_
df.head()

In [ ]:
df['grade'].hist(figsize = (10, 5))

In [ ]:
def balance_data(class_size, df):
    train_df = df.groupby(['grade']).apply(lambda x: x.sample(class_size, replace = True)).reset_index(drop = True)
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    print('New Data Size:', train_df.shape[0], 'Old Size:', df.shape[0])
    train_df['grade'].hist(figsize = (10, 5))
    return train_df

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df,test_size=0.1) # Here we will perform an 90%/10% split of the dataset, with stratification to keep similar distribution in validation set

In [ ]:
train_df['grade'].hist(figsize = (10, 5))
len(train_df)

In [ ]:
train_df.pivot_table(index='grade', aggfunc=len)

In [ ]:
train_df = balance_data(train_df.pivot_table(index='grade', aggfunc=len).max().max(),train_df) # I will oversample such that all classes have the same number of images as the maximum
train_df['grade'].hist(figsize = (10, 5))
print(len(train_df))

# Data Generator

In [ ]:
bs = 16

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=5)
train_generator = train_datagen.flow_from_dataframe(
    train_df[['filename','class']],
    "../input/aptos2019-blindness-detection/train_images",
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical')

# Model (ResNet50)

In [ ]:
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.models import Model
restnet = ResNet50(include_top=False,weights=None,input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input,output)

In [ ]:
restnet.summary()

In [ ]:
restnet.trainable = True
set_trainable = False
for layer in restnet.layers:
    if layer.name in ['conv5_block3_2_conv','conv5_block3_3_conv' , 'conv5_block3_out']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in restnet.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']).tail(10)

In [ ]:
input_shape=224,224,3
model_finetuned = Sequential()#
model_finetuned.add(restnet)
model_finetuned.add(Dense(256, activation='relu'))
model_finetuned.add(Dropout(0.3))
model_finetuned.add(Dense(256, activation='relu'))
model_finetuned.add(Dropout(0.3))
model_finetuned.add(Dense(5, activation='softmax'))
#model_finetuned.compile(loss='categorical_crossentropy',
            #  optimizer=keras.optimizers.RMSprop(lr=1e-5),
             # metrics=['accuracy'])

# Model Compile and Fit

In [ ]:
random.seed = 2
np.random.seed = 2
tf.seed = 2
tf.random.set_seed(2)

#model = Resnet50(input_shape = (224, 224, 3), classes = 5)
opt = keras.optimizers.Adam(learning_rate = 0.00008, beta_1 = 0.9)
model_finetuned.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

early_stopping= tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', min_delta = 0.01, patience = 10)

#train_generator = data_gen(train_df, enc)
# val_generator = data_gen(val_df, enc)

hist = model_finetuned.fit(train_generator,
                 steps_per_epoch = int(len(train_df)/bs),
                 epochs = 25, 
#                  validation_data = val_generator, 
#                  validation_steps = int(len(val_df) / bs),
                 callbacks = [early_stopping], 
                 shuffle = True)

print('Done')

In [ ]:
#serialize and save model 
# serialize model to JSON
model_json = model.to_json()
with open("model_{}.json".format('gradingNew11'), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_{}.h5".format('gradingNew11'))
print("Saved model to disk")

In [ ]:
plt.plot(hist.history['loss'])